In [1]:
import numpy as np
import pandas as pd
import string
import folium
import ipinfo
import requests
import subprocess

In [7]:
access_token = '4d6d9281a69d39'
handler = ipinfo.getHandler(access_token)

def get_ip_publica():
    r = requests.get('https://api.ipify.org?format=json')
    ip_publica = r.json()['ip']
    #print('Pública: ', ip_publica)
    return ip_publica

def ip_to_geo(ip_address):
    #print(ip_address)
    details = handler.getDetails(ip_address)
    lat_long = [details.all['latitude'], details.all['longitude']]
    return lat_long

def plot_traceroute(lista_ip):
    locations = []
    for ip in lista_ip:
        #print(ip)
        # Exceptiones
        exc_host = '.net' or '.com' or '.ar'
        if '10.' in ip or exc_host in ip:
            continue
        else:
            lat_long = ip_to_geo(ip)
            if not None in lat_long:
                locations.append(lat_long)
    #print(locations)
    center_lat, center_lon = locations[0][0], locations[0][1]
    p1 = [locations[0][0], locations[0][1]]
    p2 = [locations[1][0], locations[1][1]]
    my_map = folium.Map(location=[center_lat, center_lon], zoom_start=1)
    #add markers
    for each in locations:
        each[0], each[1] = float(each[0]), float(each[1])
        folium.Marker(each).add_to(my_map)
        #add lines
        folium.PolyLine(locations, color="red", weight=2.5, opacity=1).add_to(my_map)
    return my_map

def ip_to_map(ip_dst='', url=''):
    if url:
        out_host = str(subprocess.check_output(['host', url]))
        ip_dst = out_host.split(' ')[3].split('\\')[0]
        out_traceroute = str(subprocess.check_output(['traceroute', '-n', ip_dst]))
    else:
        out_traceroute = str(subprocess.check_output(['traceroute', '-n', ip_dst]))
        #print(out_traceroute)
    lista_ip = [get_ip_publica()]
    for linea in out_traceroute.split('\\n')[2:-1]:
        linea_spliteada = linea.split(' ')[3]
        if not '*' in linea_spliteada or '10' in linea_spliteada:
            ip_address = linea_spliteada
            lista_ip.append(ip_address)
    return plot_traceroute(lista_ip)

In [11]:
ip_to_map(url='www.gov.br')